# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

![](DEND-Porject-Page-2.png)

In [2]:
import pandas as pd
import configparser
from datetime import datetime
import os

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, to_timestamp, datediff,  month, year
from pyspark.sql.types import TimestampType, DateType, IntegerType, DoubleType

config = configparser.ConfigParser()
config.read('dl.cfg')

aws_key = os.environ['AWS_ACCESS_KEY_ID']=config.get('CREDENTIALS','AWS_ACCESS_KEY_ID')
aws_secret = os.environ['AWS_SECRET_ACCESS_KEY']=config.get('CREDENTIALS','AWS_SECRET_ACCESS_KEY')

spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11,org.apache.hadoop:hadoop-aws:2.7.0")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [7]:
import boto3

s3 = boto3.client('s3')
data = s3.get_object(Bucket='dend-capstone', Key='/us_ports/us_ports.json')
contents = data['Body'].read()
print(contents)
    
# with open("s3a://dend-capstone/us_ports/us_ports.json") as json_file:
#     us_states = json.load(json_file)

NoSuchKey: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.

In [8]:
df_spark.write.parquet("s3a://dend-capstone/raw_data/immigration", mode="overwrite")
#df_spark=spark.read.parquet("sas_data")

Py4JJavaError: An error occurred while calling o133.parquet.
: com.amazonaws.AmazonClientException: Unable to load AWS credentials from any provider in the chain
	at com.amazonaws.auth.AWSCredentialsProviderChain.getCredentials(AWSCredentialsProviderChain.java:117)
	at com.amazonaws.services.s3.AmazonS3Client.invoke(AmazonS3Client.java:3521)
	at com.amazonaws.services.s3.AmazonS3Client.headBucket(AmazonS3Client.java:1031)
	at com.amazonaws.services.s3.AmazonS3Client.doesBucketExist(AmazonS3Client.java:994)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(S3AFileSystem.java:297)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2669)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:94)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2703)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2685)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:373)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at 

In [219]:
df_spark.count()

3096313

In [220]:
df_spark.show(5)

+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|    cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|5748517.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     CA|20582.0|  40.0|    1.0|  1.0|20160430|     SYD| null|      G|      O|   null|      M| 1976.0|10292016|     F|  null|     QF|9.495387003E10|00011|      B1|
|5748518.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     NV|20591.0|  32.0|    1.0|  

In [221]:
df_spark = df_spark.select(['cicid', 'i94yr', 'i94mon', 'i94res', 'i94port', 'arrdate', 'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'gender', 'airline', 'visatype'])

In [222]:
df_spark.show(5)

+---------+------+------+------+-------+-------+-------+-------+-------+------+-------+------+-------+--------+
|    cicid| i94yr|i94mon|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|gender|airline|visatype|
+---------+------+------+------+-------+-------+-------+-------+-------+------+-------+------+-------+--------+
|5748517.0|2016.0|   4.0| 438.0|    LOS|20574.0|    1.0|     CA|20582.0|  40.0|    1.0|     F|     QF|      B1|
|5748518.0|2016.0|   4.0| 438.0|    LOS|20574.0|    1.0|     NV|20591.0|  32.0|    1.0|     F|     VA|      B1|
|5748519.0|2016.0|   4.0| 438.0|    LOS|20574.0|    1.0|     WA|20582.0|  29.0|    1.0|     M|     DL|      B1|
|5748520.0|2016.0|   4.0| 438.0|    LOS|20574.0|    1.0|     WA|20588.0|  29.0|    1.0|     F|     DL|      B1|
|5748521.0|2016.0|   4.0| 438.0|    LOS|20574.0|    1.0|     WA|20588.0|  28.0|    1.0|     M|     DL|      B1|
+---------+------+------+------+-------+-------+-------+-------+-------+------+-------+------+-------+--

In [223]:
df_spark.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- visatype: string (nullable = true)



In [224]:
df_spark = df_spark.withColumn("cicid", df_spark["cicid"].cast(IntegerType())) \
                    .withColumn("i94yr", df_spark["i94yr"].cast(IntegerType())) \
                    .withColumn("i94mon", df_spark["i94mon"].cast(IntegerType())) \
                    .withColumn("i94res", df_spark["i94res"].cast(IntegerType())) \
                    .withColumn("arrdate", df_spark["arrdate"].cast(IntegerType())) \
                    .withColumn("i94mode", df_spark["i94mode"].cast(IntegerType())) \
                    .withColumn("depdate", df_spark["depdate"].cast(IntegerType())) \
                    .withColumn("i94bir", df_spark["i94bir"].cast(IntegerType())) \
                    .withColumn("i94visa", df_spark["i94visa"].cast(IntegerType())) \

In [225]:
df_spark.printSchema()

root
 |-- cicid: integer (nullable = true)
 |-- i94yr: integer (nullable = true)
 |-- i94mon: integer (nullable = true)
 |-- i94res: integer (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: integer (nullable = true)
 |-- i94mode: integer (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: integer (nullable = true)
 |-- i94bir: integer (nullable = true)
 |-- i94visa: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- visatype: string (nullable = true)



In [228]:
df_spark.show(5)

+-------+-----+------+------+-------+-------+-------+-------+-------+------+-------+------+-------+--------+
|  cicid|i94yr|i94mon|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|gender|airline|visatype|
+-------+-----+------+------+-------+-------+-------+-------+-------+------+-------+------+-------+--------+
|5748517| 2016|     4|   438|    LOS|  20574|      1|     CA|  20582|    40|      1|     F|     QF|      B1|
|5748518| 2016|     4|   438|    LOS|  20574|      1|     NV|  20591|    32|      1|     F|     VA|      B1|
|5748519| 2016|     4|   438|    LOS|  20574|      1|     WA|  20582|    29|      1|     M|     DL|      B1|
|5748520| 2016|     4|   438|    LOS|  20574|      1|     WA|  20588|    29|      1|     F|     DL|      B1|
|5748521| 2016|     4|   438|    LOS|  20574|      1|     WA|  20588|    28|      1|     M|     DL|      B1|
+-------+-----+------+------+-------+-------+-------+-------+-------+------+-------+------+-------+--------+
only showing top 5 

In [229]:
# https://stackoverflow.com/questions/44413132/count-the-number-of-missing-values-in-a-dataframe-spark
from pyspark.sql.functions import col,sum
Null_stats = df_spark.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df_spark.columns))

In [230]:
Null_stats.show()

+-----+-----+------+------+-------+-------+-------+-------+-------+------+-------+------+-------+--------+
|cicid|i94yr|i94mon|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|gender|airline|visatype|
+-----+-----+------+------+-------+-------+-------+-------+-------+------+-------+------+-------+--------+
|    0|    0|     0|     0|      0|      0|    239| 152592| 142457|   802|      0|414269|  83627|       0|
+-----+-----+------+------+-------+-------+-------+-------+-------+------+-------+------+-------+--------+



In [231]:
# drop all nulls
df_spark = df_spark.na.drop()

In [232]:
df_spark.count()

2384915

In [233]:
new_Null_stats = df_spark.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df_spark.columns)).show()

+-----+-----+------+------+-------+-------+-------+-------+-------+------+-------+------+-------+--------+
|cicid|i94yr|i94mon|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|gender|airline|visatype|
+-----+-----+------+------+-------+-------+-------+-------+-------+------+-------+------+-------+--------+
|    0|    0|     0|     0|      0|      0|      0|      0|      0|     0|      0|     0|      0|       0|
+-----+-----+------+------+-------+-------+-------+-------+-------+------+-------+------+-------+--------+



In [234]:
df_spark.show(5)

+-------+-----+------+------+-------+-------+-------+-------+-------+------+-------+------+-------+--------+
|  cicid|i94yr|i94mon|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|gender|airline|visatype|
+-------+-----+------+------+-------+-------+-------+-------+-------+------+-------+------+-------+--------+
|5748517| 2016|     4|   438|    LOS|  20574|      1|     CA|  20582|    40|      1|     F|     QF|      B1|
|5748518| 2016|     4|   438|    LOS|  20574|      1|     NV|  20591|    32|      1|     F|     VA|      B1|
|5748519| 2016|     4|   438|    LOS|  20574|      1|     WA|  20582|    29|      1|     M|     DL|      B1|
|5748520| 2016|     4|   438|    LOS|  20574|      1|     WA|  20588|    29|      1|     F|     DL|      B1|
|5748521| 2016|     4|   438|    LOS|  20574|      1|     WA|  20588|    28|      1|     M|     DL|      B1|
+-------+-----+------+------+-------+-------+-------+-------+-------+------+-------+------+-------+--------+
only showing top 5 

In [235]:
df_spark.select('i94addr').distinct().show(10)

+-------+
|i94addr|
+-------+
|     .N|
|     CI|
|     TC|
|     SC|
|     AZ|
|     NS|
|     SL|
|     LA|
|     NL|
|     MN|
+-------+
only showing top 10 rows



In [31]:
i94addr_df  = pd.read_json('raw_data/I94addr.json', typ='series').to_frame("state").reset_index().rename(columns={"index": "code"})

In [1]:
#i94addr_df.head()

In [238]:
ls = list(i94addr_df['code'])

In [239]:
valid_state = udf(lambda x: '99' if x not in list(i94addr_df['code']) else x)


In [240]:
df_spark = df_spark.withColumn("i94addr", valid_state(df_spark.i94addr))

In [241]:
df_spark.select('i94addr').distinct().show(10)

+-------+
|i94addr|
+-------+
|     SC|
|     AZ|
|     LA|
|     MN|
|     NJ|
|     DC|
|     OR|
|     99|
|     VA|
|     RI|
+-------+
only showing top 10 rows



In [242]:
df_spark.show(5)

+-------+-----+------+------+-------+-------+-------+-------+-------+------+-------+------+-------+--------+
|  cicid|i94yr|i94mon|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|gender|airline|visatype|
+-------+-----+------+------+-------+-------+-------+-------+-------+------+-------+------+-------+--------+
|5748517| 2016|     4|   438|    LOS|  20574|      1|     CA|  20582|    40|      1|     F|     QF|      B1|
|5748518| 2016|     4|   438|    LOS|  20574|      1|     NV|  20591|    32|      1|     F|     VA|      B1|
|5748519| 2016|     4|   438|    LOS|  20574|      1|     WA|  20582|    29|      1|     M|     DL|      B1|
|5748520| 2016|     4|   438|    LOS|  20574|      1|     WA|  20588|    29|      1|     F|     DL|      B1|
|5748521| 2016|     4|   438|    LOS|  20574|      1|     WA|  20588|    28|      1|     M|     DL|      B1|
+-------+-----+------+------+-------+-------+-------+-------+-------+------+-------+------+-------+--------+
only showing top 5 

In [243]:
df_spark.select('i94addr').distinct().show(20)

+-------+
|i94addr|
+-------+
|     SC|
|     AZ|
|     LA|
|     MN|
|     NJ|
|     DC|
|     OR|
|     99|
|     VA|
|     RI|
|     KY|
|     WY|
|     NH|
|     MI|
|     NV|
|     WI|
|     ID|
|     CA|
|     CT|
|     NE|
+-------+
only showing top 20 rows



In [244]:
df_spark.select('i94mode').distinct().show(10)

+-------+
|i94mode|
+-------+
|      1|
|      3|
|      9|
|      2|
+-------+



In [245]:
df_spark.select('gender').distinct().show(10)

+------+
|gender|
+------+
|     F|
|     M|
|     U|
|     X|
+------+



In [246]:
df_spark.show(5)

+-------+-----+------+------+-------+-------+-------+-------+-------+------+-------+------+-------+--------+
|  cicid|i94yr|i94mon|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|gender|airline|visatype|
+-------+-----+------+------+-------+-------+-------+-------+-------+------+-------+------+-------+--------+
|5748517| 2016|     4|   438|    LOS|  20574|      1|     CA|  20582|    40|      1|     F|     QF|      B1|
|5748518| 2016|     4|   438|    LOS|  20574|      1|     NV|  20591|    32|      1|     F|     VA|      B1|
|5748519| 2016|     4|   438|    LOS|  20574|      1|     WA|  20582|    29|      1|     M|     DL|      B1|
|5748520| 2016|     4|   438|    LOS|  20574|      1|     WA|  20588|    29|      1|     F|     DL|      B1|
|5748521| 2016|     4|   438|    LOS|  20574|      1|     WA|  20588|    28|      1|     M|     DL|      B1|
+-------+-----+------+------+-------+-------+-------+-------+-------+------+-------+------+-------+--------+
only showing top 5 

In [247]:
# let's convert SAS date format to date .. I follow :
#https://stackoverflow.com/questions/36412864/convert-numeric-sas-date-to-datetime-in-pandas
#https://communities.sas.com/t5/Developers/How-can-I-read-a-SAS-datetime-value-in-Python/td-p/433593#
# from pyspark.sql.types import DateType
# from pyspark.sql.functions import udf, col, to_timestamp

sasdate_to_date = udf(lambda x: ( pd.to_timedelta(x, unit='D') + pd.Timestamp('1960-1-1') ).date(), DateType() )

In [248]:
df_spark = df_spark.withColumn("arrdate", sasdate_to_date(df_spark.arrdate)) \
                    .withColumn("depdate", sasdate_to_date(df_spark.depdate))

In [249]:
df_spark.show(5)

+-------+-----+------+------+-------+----------+-------+-------+----------+------+-------+------+-------+--------+
|  cicid|i94yr|i94mon|i94res|i94port|   arrdate|i94mode|i94addr|   depdate|i94bir|i94visa|gender|airline|visatype|
+-------+-----+------+------+-------+----------+-------+-------+----------+------+-------+------+-------+--------+
|5748517| 2016|     4|   438|    LOS|2016-04-30|      1|     CA|2016-05-08|    40|      1|     F|     QF|      B1|
|5748518| 2016|     4|   438|    LOS|2016-04-30|      1|     NV|2016-05-17|    32|      1|     F|     VA|      B1|
|5748519| 2016|     4|   438|    LOS|2016-04-30|      1|     WA|2016-05-08|    29|      1|     M|     DL|      B1|
|5748520| 2016|     4|   438|    LOS|2016-04-30|      1|     WA|2016-05-14|    29|      1|     F|     DL|      B1|
|5748521| 2016|     4|   438|    LOS|2016-04-30|      1|     WA|2016-05-14|    28|      1|     M|     DL|      B1|
+-------+-----+------+------+-------+----------+-------+-------+----------+-----

In [250]:
from pyspark.sql.functions import datediff
df_spark = df_spark.withColumn("stay_length", datediff(df_spark.depdate, df_spark.arrdate) )

In [251]:
df_spark.show(5)

+-------+-----+------+------+-------+----------+-------+-------+----------+------+-------+------+-------+--------+-----------+
|  cicid|i94yr|i94mon|i94res|i94port|   arrdate|i94mode|i94addr|   depdate|i94bir|i94visa|gender|airline|visatype|stay_length|
+-------+-----+------+------+-------+----------+-------+-------+----------+------+-------+------+-------+--------+-----------+
|5748517| 2016|     4|   438|    LOS|2016-04-30|      1|     CA|2016-05-08|    40|      1|     F|     QF|      B1|          8|
|5748518| 2016|     4|   438|    LOS|2016-04-30|      1|     NV|2016-05-17|    32|      1|     F|     VA|      B1|         17|
|5748519| 2016|     4|   438|    LOS|2016-04-30|      1|     WA|2016-05-08|    29|      1|     M|     DL|      B1|          8|
|5748520| 2016|     4|   438|    LOS|2016-04-30|      1|     WA|2016-05-14|    29|      1|     F|     DL|      B1|         14|
|5748521| 2016|     4|   438|    LOS|2016-04-30|      1|     WA|2016-05-14|    28|      1|     M|     DL|      

In [252]:
from pyspark.sql.functions import when
df_spark = df_spark.withColumn("i94mode",when(df_spark.i94mode == 1, 'Air')\
                               .when(df_spark.i94mode == 2, 'Sea')\
                               .when(df_spark.i94mode == 3, 'Land')\
                               .otherwise('other'))

  
#df_spark = df_spark.withColumn("i94mode", df_spark["i94mode"].cast(IntegerType()))

In [253]:
df_spark = df_spark.withColumn("i94visa",when(df_spark.i94visa == 1, 'Business')\
                               .when(df_spark.i94visa == 2, 'Pleasure')\
                               .when(df_spark.i94visa == 3, 'Student')\
                               .otherwise('other'))  

In [255]:
df_spark.select('i94mode').distinct().show(10)

+-------+
|i94mode|
+-------+
|    Sea|
|  other|
|   Land|
|    Air|
+-------+



In [259]:
df_spark.show(5)

+-------+-----+------+------+-------+----------+-------+-------+----------+------+--------+------+-------+--------+-----------+
|  cicid|i94yr|i94mon|i94res|i94port|   arrdate|i94mode|i94addr|   depdate|i94bir| i94visa|gender|airline|visatype|stay_length|
+-------+-----+------+------+-------+----------+-------+-------+----------+------+--------+------+-------+--------+-----------+
|5748517| 2016|     4|   438|    LOS|2016-04-30|    Air|     CA|2016-05-08|    40|Business|     F|     QF|      B1|          8|
|5748518| 2016|     4|   438|    LOS|2016-04-30|    Air|     NV|2016-05-17|    32|Business|     F|     VA|      B1|         17|
|5748519| 2016|     4|   438|    LOS|2016-04-30|    Air|     WA|2016-05-08|    29|Business|     M|     DL|      B1|          8|
|5748520| 2016|     4|   438|    LOS|2016-04-30|    Air|     WA|2016-05-14|    29|Business|     F|     DL|      B1|         14|
|5748521| 2016|     4|   438|    LOS|2016-04-30|    Air|     WA|2016-05-14|    28|Business|     M|     D

## Step 2: Explore and Assess the Data

### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### I94 Immigration Data

- I'll use the provided dataset sample in csv format to explore I94 Immigration data, and make some assumtions before working with the full dataset in Parquet format.
- Using the dataset sample I'll determine the needed columns, and the required cleansing steps, then apply that on the full parquet dataset.

In [7]:
pd.set_option('display.max_columns', 500)
img_df = pd.read_csv("immigration_data_sample.csv")

In [8]:
img_df.columns

Index(['Unnamed: 0', 'cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port',
       'arrdate', 'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa',
       'count', 'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd',
       'entdepu', 'matflag', 'biryear', 'dtaddto', 'gender', 'insnum',
       'airline', 'admnum', 'fltno', 'visatype'],
      dtype='object')

In [9]:
img_df.head(10)

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,NaN,NaN,G,O,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,NaN,G,R,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,NaN,NaN,G,O,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,NaN,G,O,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,NaN,NaN,Z,K,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT
5,721257,1481650.0,2016.0,4.0,577.0,577.0,ATL,20552.0,1.0,GA,20606.0,51.0,2.0,1.0,20160408,NaN,NaN,T,N,NaN,M,1965.0,10072016,M,NaN,DL,7.368526e+08,910,B2
6,1072780,2197173.0,2016.0,4.0,245.0,245.0,SFR,20556.0,1.0,CA,20635.0,48.0,2.0,1.0,20160412,NaN,NaN,T,O,NaN,M,1968.0,10112016,F,NaN,CX,7.863122e+08,870,B2
7,112205,232708.0,2016.0,4.0,113.0,135.0,NYC,20546.0,1.0,NY,20554.0,33.0,2.0,1.0,20160402,NaN,NaN,G,O,NaN,M,1983.0,06302016,F,NaN,BA,5.547449e+10,00117,WT
8,2577162,5227851.0,2016.0,4.0,131.0,131.0,CHI,20572.0,1.0,IL,20575.0,39.0,2.0,1.0,20160428,NaN,NaN,O,O,NaN,M,1977.0,07262016,NaN,NaN,LX,5.941342e+10,00008,WT
9,10930,13213.0,2016.0,4.0,116.0,116.0,LOS,20545.0,1.0,CA,20553.0,35.0,2.0,1.0,20160401,NaN,NaN,O,O,NaN,M,1981.0,06292016,NaN,NaN,AA,5.544979e+10,00109,WT


In [10]:
valid_cloumns = [ 'cicid', 'i94yr', 'i94mon', 'i94res', 'i94port', 'arrdate', 'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'gender', 'airline', 'visatype']
invalid_columns = ['Unnamed: 0','i94cit', 'arrdate','count',  'dtadfile', 'visapost', 'occup','entdepa', 'entdepd','entdepu','matflag', 'biryear','dtaddto', 'insnum','admnum', 'fltno' ]

In [11]:
img_df = img_df[['cicid', 'i94yr', 'i94mon', 'i94res', 'i94port', 'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'gender', 'airline', 'visatype']]

In [15]:
img_df.head(5)

,cicid,i94yr,i94mon,i94res,i94port,i94mode,i94addr,depdate,i94bir,i94visa,gender,airline,visatype
0,4084316.0,2016.0,4.0,209.0,HHW,1.0,HI,20573.0,61.0,2.0,F,JL,WT
1,4422636.0,2016.0,4.0,582.0,MCA,1.0,TX,20568.0,26.0,2.0,M,*GA,B2
2,1195600.0,2016.0,4.0,112.0,OGG,1.0,FL,20571.0,76.0,2.0,M,LH,WT
3,5291768.0,2016.0,4.0,297.0,LOS,1.0,CA,20581.0,25.0,2.0,M,QR,B2
4,985523.0,2016.0,4.0,111.0,CHM,3.0,NY,20553.0,19.0,2.0,F,NaN,WT


#### Airline IATA Codes

In [5]:
#https://github.com/BesrourMS/Airlines
#i94addr_df  = pd.read_json('raw_data/I94addr.json', typ='series').to_frame("state").reset_index().rename(columns={"index": "code"})
# airline_df  = pd.read_json('raw_date/airlines_iata_codes.json', typ='series')
# airline_df.head()

In [34]:
import pandas as pd
airline_df  = pd.read_json('raw_data/airlines_iata_codes.json')
airline_df.head()

,iata,name
0,0A,Amber Air
1,0B,Blue Air
2,0D,Darwin Airline
3,2B,Aerocondor
4,2C,SNCF


#### World Temperature Data

In [2]:
spark_tmp = spark.read.csv('raw_data/GlobalLandTemperaturesByState.csv', header = True)

In [3]:
spark_tmp.count()

645675

In [4]:
spark_tmp.show(5)

+----------+------------------+-----------------------------+-----+-------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|State|Country|
+----------+------------------+-----------------------------+-----+-------+
|1855-05-01|            25.544|                        1.171| Acre| Brazil|
|1855-06-01|            24.228|                        1.103| Acre| Brazil|
|1855-07-01|            24.371|                        1.044| Acre| Brazil|
|1855-08-01|            25.427|                        1.073| Acre| Brazil|
|1855-09-01|            25.675|                        1.014| Acre| Brazil|
+----------+------------------+-----------------------------+-----+-------+
only showing top 5 rows



In [5]:
spark_tmp = spark_tmp.withColumn("dt", spark_tmp["dt"].cast(DateType())) \
                        .withColumnRenamed("dt", "date")

In [6]:
spark_tmp = spark_tmp.filter((spark_tmp.Country == 'United States') & (spark_tmp.date >= '2011-01-01')).dropDuplicates()

In [7]:
spark_tmp.show(5)

+----------+------------------+-----------------------------+--------------+-------------+
|      date|AverageTemperature|AverageTemperatureUncertainty|         State|      Country|
+----------+------------------+-----------------------------+--------------+-------------+
|2012-10-01|            17.541|                        0.236|       Arizona|United States|
|2012-03-01|            21.678|                        0.211|       Florida|United States|
|2011-01-01|             -5.21|                         0.29|         Idaho|United States|
|2013-07-01|            20.621|                         0.29|      Michigan|United States|
|2012-03-01|            17.774|                         0.16|South Carolina|United States|
+----------+------------------+-----------------------------+--------------+-------------+
only showing top 5 rows



In [25]:
from pyspark.sql.functions import year, month
spark_tmp = spark_tmp.select(['date', 'AverageTemperature', 'State']) \
.withColumn("month", month(spark_tmp["date"])) \
.withColumn("year", year(spark_tmp["date"]))

In [26]:
spark_tmp.show(5)

+----------+------------------+--------------+-----+----+
|      date|AverageTemperature|         State|month|year|
+----------+------------------+--------------+-----+----+
|2012-10-01|            17.541|       Arizona|   10|2012|
|2012-03-01|            21.678|       Florida|    3|2012|
|2011-01-01|             -5.21|         Idaho|    1|2011|
|2013-07-01|            20.621|      Michigan|    7|2013|
|2012-03-01|            17.774|South Carolina|    3|2012|
+----------+------------------+--------------+-----+----+
only showing top 5 rows



In [28]:
from pyspark.sql.functions import udf, col, to_timestamp, datediff, when, month, year
spark_tmp = spark_tmp.withColumn("State",when(spark_tmp.State == 'Georgia (State)', 'Georgia').otherwise(spark_tmp.State))

In [31]:
import json
with open('raw_data/I94addr.json') as json_file:
    us_states = json.load(json_file)

In [32]:
state_abbr = udf( lambda x: {k.upper(): v.upper() for k, v in us_states.items()}[x.upper()])

In [33]:
spark_tmp.show(5)

+----------+------------------+--------------+-----+----+
|      date|AverageTemperature|         State|month|year|
+----------+------------------+--------------+-----+----+
|2012-10-01|            17.541|       Arizona|   10|2012|
|2012-03-01|            21.678|       Florida|    3|2012|
|2011-01-01|             -5.21|         Idaho|    1|2011|
|2013-07-01|            20.621|      Michigan|    7|2013|
|2012-03-01|            17.774|South Carolina|    3|2012|
+----------+------------------+--------------+-----+----+
only showing top 5 rows



In [35]:
spark_tmp = spark_tmp.withColumn("State",  state_abbr(spark_tmp.State))

In [36]:
spark_tmp.show()

+----------+------------------+-----+-----+----+
|      date|AverageTemperature|State|month|year|
+----------+------------------+-----+-----+----+
|2012-10-01|            17.541|   AZ|   10|2012|
|2012-03-01|            21.678|   FL|    3|2012|
|2011-01-01|             -5.21|   ID|    1|2011|
|2013-07-01|            20.621|   MI|    7|2013|
|2012-03-01|            17.774|   SC|    3|2012|
|2012-10-01| 7.752999999999999|   SD|   10|2012|
|2013-08-01|             24.06|   TN|    8|2013|
|2011-01-01|            -1.169|   DE|    1|2011|
|2013-04-01|             4.252|   ID|    4|2013|
|2013-06-01|            22.248|   IN|    6|2013|
|2012-11-01|0.8579999999999999|   ME|   11|2012|
|2011-08-01|29.011000000000006|   MS|    8|2011|
|2013-07-01|            23.124|   NM|    7|2013|
|2012-03-01|              3.86|   OR|    3|2012|
|2012-06-01|            24.301|   SC|    6|2012|
|2012-08-01|            20.008|   WI|    8|2012|
|2012-07-01|            26.629|   DC|    7|2012|
|2011-06-01|21.69199

In [126]:
spark_tmp.count()

1683

In [192]:
spark.stop()

#### Airports Data

In [39]:
df_spark = spark.read.csv('raw_data/airport-codes_csv.csv', header = True)

In [40]:
spark_air.count()

55075

In [48]:
spark_air.show(5)

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
| 01AK|     heliport|Providence Seward...|         120|       NA|         US|     US-AK|      Seward|    01AK|     null|      01AK|-149.446249008, 6...|
| 04WA|small_airport|  Ox Meadows Airport|        2345|       NA|         US|     US-WA|     Spokane|    04WA|     null|      04WA|-117.43699646, 47...|
| 08OR|small_airport| Saxon Sycan Airport|        4990|       NA|         US|     US-OR| Silver Lake|    08OR|     null|      08OR|-121.116996765136...|
| 0CD9|     heliport|Swedish S.W. Medi...|        5563|       NA|         US|     

In [50]:
# columns
df_spark = df_spark.filter(df_spark.iso_country == 'US').select(['ident', 'type', 'name', 'elevation_ft', 'iso_region', 'municipality', 'iata_code', 'local_code'])

In [51]:
spark_air.show(5)

+-----+-------------+--------------------+------------+----------+------------+---------+----------+
|ident|         type|                name|elevation_ft|iso_region|municipality|iata_code|local_code|
+-----+-------------+--------------------+------------+----------+------------+---------+----------+
| 01AK|     heliport|Providence Seward...|         120|     US-AK|      Seward|     null|      01AK|
| 04WA|small_airport|  Ox Meadows Airport|        2345|     US-WA|     Spokane|     null|      04WA|
| 08OR|small_airport| Saxon Sycan Airport|        4990|     US-OR| Silver Lake|     null|      08OR|
| 0CD9|     heliport|Swedish S.W. Medi...|        5563|     US-CO|   Littleton|     null|      0CD9|
| 0GA3|small_airport|   Ayresouth Airport|        1287|     US-GA|      Temple|     null|      0GA3|
+-----+-------------+--------------------+------------+----------+------------+---------+----------+
only showing top 5 rows



In [52]:
spark_air.count()

22757

In [83]:
spark_air.select('type').distinct().show(10)

+--------------+
|          type|
+--------------+
| large_airport|
|   balloonport|
| seaplane_base|
|      heliport|
|        closed|
|medium_airport|
| small_airport|
+--------------+



In [59]:
#spark_air.filter(spark_air.type == 'small_airport').show(10)
#string[-5:]

In [78]:
spark_air = spark_air.withColumn("iso_region", spark_air.iso_region[-2:2])

In [81]:
#spark_air.select('iso_region').distinct().show(70)

In [82]:
spark_air.show(5)

+-----+-------------+--------------------+------------+----------+------------+---------+----------+
|ident|         type|                name|elevation_ft|iso_region|municipality|iata_code|local_code|
+-----+-------------+--------------------+------------+----------+------------+---------+----------+
| 01AK|     heliport|Providence Seward...|         120|        AK|      Seward|     null|      01AK|
| 04WA|small_airport|  Ox Meadows Airport|        2345|        WA|     Spokane|     null|      04WA|
| 08OR|small_airport| Saxon Sycan Airport|        4990|        OR| Silver Lake|     null|      08OR|
| 0CD9|     heliport|Swedish S.W. Medi...|        5563|        CO|   Littleton|     null|      0CD9|
| 0GA3|small_airport|   Ayresouth Airport|        1287|        GA|      Temple|     null|      0GA3|
+-----+-------------+--------------------+------------+----------+------------+---------+----------+
only showing top 5 rows



In [85]:
#spark_air.filter(spark_air.type == 'large_airport').show(10)

In [86]:
spark_air.printSchema()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)



In [88]:
spark_air = spark_air.withColumn("elevation_ft", spark_air["elevation_ft"].cast(IntegerType()))

In [89]:
spark_air.printSchema()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)



#### U.S. City Demographic Data

In [244]:
dm_spark = spark.read.csv('raw_data/us-cities-demographics.csv', sep = ';', header = True)

In [245]:
dm_spark.count()

2891

In [246]:
dm_spark.show(5)

+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|            City|        State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|                Race|Count|
+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|   Silver Spring|     Maryland|      33.8|          40601|            41862|           82463|              1562|       30908|                   2.6|        MD|  Hispanic or Latino|25924|
|          Quincy|Massachusetts|      41.0|          44129|            49500|           93629|              4147|       32935|                  2.39|        MA|               White|58723|
|          Hoover|      Alabama|      38.5|          38040| 

In [247]:
#dm_spark.printSchema()

In [248]:
df_spark = df_spark.withColumnRenamed("Median Age", "Median_Age")\
                    .withColumnRenamed("Male Population", "Male_Population")\
                    .withColumnRenamed("Female Population", "Female_Population")\
                    .withColumnRenamed("Total Population", "Total_Population")\
                    .withColumnRenamed("Foreign-born", "Foreign_born")\
                    .withColumnRenamed("State Code", "State_Code")\
                    .withColumn("Count", df_spark["Count"].cast(IntegerType())) \
                    .select(['City', 'State', 'Median_Age', 'Male_Population',\
                             'Female_Population', 'Total_Population', 'Foreign_born', 'State_Code',\
                             'Race', 'Count'])

In [249]:
#dm_spark.show(5)

In [250]:
from pyspark.sql.types import TimestampType, DateType, DoubleType

In [251]:
df_spark = df_spark.withColumn("Median_Age", df_spark["Median_Age"].cast(DoubleType())) \
                    .withColumn("Male_Population", df_spark["Male_Population"].cast(IntegerType())) \
                    .withColumn("Female_Population", df_spark["Female_Population"].cast(IntegerType())) \
                    .withColumn("Total_Population", df_spark["Total_Population"].cast(IntegerType())) \
                    .withColumn("Foreign_born", df_spark["Foreign_born"].cast(IntegerType())) \
                    .withColumn("Count", df_spark["Count"].cast(IntegerType()))

In [252]:
#dm_spark.filter((dm_spark.State_Code == 'MD') & (dm_spark.City == 'Gaithersburg')).show(50)

In [179]:
#dm_spark.filter((dm_spark.State_Code == 'MD') & ( (dm_spark.City == 'Baltimore') | (dm_spark.City == 'Gaithersburg') )).show(50)

In [191]:
#tm = dm_spark.filter((dm_spark.State_Code == 'MD') & ( (dm_spark.City == 'Baltimore') | (dm_spark.City == 'Gaithersburg') ))

In [253]:
#tm = dm_spark.filter((dm_spark.State_Code == 'MD') )

In [254]:
#tm.show(2)

In [255]:
df_pivot = dm_spark.groupBy(['City', 'State', 'Median_Age', 'Male_Population',\
                             'Female_Population', 'Total_Population', 'Foreign_born', 'State_Code']).pivot("Race").sum("Count")

In [212]:
#df_pivot.show()

In [256]:
df_pivot = df_pivot.withColumnRenamed("American Indian and Alaska Native", "Native")\
.withColumnRenamed("Black or African-American", "Black")\
.withColumnRenamed("Hispanic or Latino", "Hispanic")

In [257]:
df_pivot.show(3)

+-----------------+----------+----------+---------------+-----------------+----------------+------------+----------+------+-----+-----+--------+-----+
|             City|     State|Median_Age|Male_Population|Female_Population|Total_Population|Foreign_born|State_Code|Native|Asian|Black|Hispanic|White|
+-----------------+----------+----------+---------------+-----------------+----------------+------------+----------+------+-----+-----+--------+-----+
|    Coral Springs|   Florida|      37.2|          63316|            66186|          129502|       38552|        FL|  1262| 6573|27225|   40817|90896|
|Arlington Heights|  Illinois|      45.9|          37627|            39369|           76996|       14630|        IL|  null| 9975| 1222|    2742|66285|
|    Santa Barbara|California|      37.8|          45068|            46784|           91852|       19441|        CA|  1560| 4964| 2013|   31102|73659|
+-----------------+----------+----------+---------------+-----------------+----------------+--

In [258]:
df_pivot.createOrReplaceTempView("pivot_table")

In [259]:
qry = spark.sql("""
        SELECT State_Code, 
        cast ( avg(Median_Age) as DECIMAL(4,2) ) as age,
        sum(Male_Population) as Male_Population,
        sum(Female_Population) as Female_Population,
        sum(Total_Population) as Total_Population, 
        sum(Foreign_born) as Foreign_born,
        sum(Native) as Native,
        sum(Asian) as Asian,
        sum(Black) as Black,
        sum(White) as White
        FROM pivot_table
        GROUP BY State_Code
""")

In [260]:
qry.show(5)

+----------+-----+-------+-------+-------+-------+------+------+------+-------+
|State_Code|  age|   male|  femal|  total|Foreign|Native| Asian| Black|  White|
+----------+-----+-------+-------+-------+-------+------+------+------+-------+
|        AZ|35.04|2227455|2272087|4499542| 682313|129708|229183|296222|3591611|
|        SC|34.18| 260944| 272713| 533657|  27744|  3705| 13355|175064| 343764|
|        LA|34.63| 626998| 673597|1300595|  83419|  8263| 38739|602377| 654578|
|        MN|35.62| 702157| 720246|1422403| 215873| 25242|151544|216731|1050239|
|        NJ|35.12| 705736| 723172|1428908| 477028| 11350|116844|452202| 615083|
+----------+-----+-------+-------+-------+-------+------+------+------+-------+
only showing top 5 rows



In [172]:
tmg = dm_spark.filter((dm_spark.State_Code == 'MD') & ( (dm_spark.City == 'Gaithersburg') ))

In [173]:
tmg.show(10)

+------------+--------+----------+---------------+-----------------+----------------+------------+----------+--------------------+-----+
|        City|   State|Median_Age|Male_Population|Female_Population|Total_Population|Foreign_born|State_Code|                Race|Count|
+------------+--------+----------+---------------+-----------------+----------------+------------+----------+--------------------+-----+
|Gaithersburg|Maryland|      35.7|          32211|            35234|           67445|       28133|        MD|               White|36639|
|Gaithersburg|Maryland|      35.7|          32211|            35234|           67445|       28133|        MD|Black or African-...|12482|
|Gaithersburg|Maryland|      35.7|          32211|            35234|           67445|       28133|        MD|  Hispanic or Latino|18721|
|Gaithersburg|Maryland|      35.7|          32211|            35234|           67445|       28133|        MD|American Indian a...|  672|
|Gaithersburg|Maryland|      35.7|       

In [167]:
tm_a = dm_spark.filter((dm_spark.State_Code == 'MD') & ( (dm_spark.City == 'Baltimore') | (dm_spark.City == 'Gaithersburg') ))\
.select(['City','Median_Age', 'Male_Population',\
                             'Female_Population', 'Total_Population', 'Foreign_born', 'State_Code',\
                             'Race', 'Count'])

In [168]:
tm_a.show(30)

+------------+----------+---------------+-----------------+----------------+------------+----------+--------------------+------+
|        City|Median_Age|Male_Population|Female_Population|Total_Population|Foreign_born|State_Code|                Race| Count|
+------------+----------+---------------+-----------------+----------------+------------+----------+--------------------+------+
|   Baltimore|      34.7|         294027|           327822|          621849|       49857|        MD|  Hispanic or Latino| 29953|
|   Baltimore|      34.7|         294027|           327822|          621849|       49857|        MD|Black or African-...|396270|
|Gaithersburg|      35.7|          32211|            35234|           67445|       28133|        MD|               White| 36639|
|Gaithersburg|      35.7|          32211|            35234|           67445|       28133|        MD|Black or African-...| 12482|
|   Baltimore|      34.7|         294027|           327822|          621849|       49857|        

In [169]:
tm_a.printSchema()

root
 |-- City: string (nullable = true)
 |-- Median_Age: string (nullable = true)
 |-- Male_Population: string (nullable = true)
 |-- Female_Population: string (nullable = true)
 |-- Total_Population: string (nullable = true)
 |-- Foreign_born: string (nullable = true)
 |-- State_Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: string (nullable = true)



In [171]:
tm_a.createOrReplaceTempView("state_table")

In [184]:
qry = spark.sql("""
        SELECT State_Code, Race, sum(Count) as countt, sum(Foreign_born) as fore, avg(Total_Population) as total, avg(Female_Population) as femal, avg(Male_Population) as male, avg(Median_Age) as age
        FROM state_table
        GROUP BY State_Code, City, Race
""")

In [185]:
qry.show(5)

+----------+------------------+--------+-------+--------+--------+--------+----+
|State_Code|              Race|  countt|   fore|   total|   femal|    male| age|
+----------+------------------+--------+-------+--------+--------+--------+----+
|        MD|Hispanic or Latino| 18721.0|28133.0| 67445.0| 35234.0| 32211.0|35.7|
|        MD|             White| 36639.0|28133.0| 67445.0| 35234.0| 32211.0|35.7|
|        MD|             Asian| 20161.0|49857.0|621849.0|327822.0|294027.0|34.7|
|        MD|Hispanic or Latino| 29953.0|49857.0|621849.0|327822.0|294027.0|34.7|
|        MD|             White|207274.0|49857.0|621849.0|327822.0|294027.0|34.7|
+----------+------------------+--------+-------+--------+--------+--------+----+
only showing top 5 rows



# Test here

In [264]:
df_spark = spark.read.csv('raw_data/us-cities-demographics.csv', sep = ';', header = True)

In [265]:
df_spark = df_spark.withColumnRenamed("Median Age", "Median_Age")\
                    .withColumnRenamed("Male Population", "Male_Population")\
                    .withColumnRenamed("Female Population", "Female_Population")\
                    .withColumnRenamed("Total Population", "Total_Population")\
                    .withColumnRenamed("Foreign-born", "Foreign_born")\
                    .withColumnRenamed("State Code", "State_Code")\
                    .withColumn("Count", df_spark["Count"].cast(IntegerType())) \
                    .select(['City', 'State', 'Median_Age', 'Male_Population',\
                             'Female_Population', 'Total_Population', 'Foreign_born', 'State_Code',\
                             'Race', 'Count'])

In [266]:
df_spark = df_spark.withColumn("Median_Age", df_spark["Median_Age"].cast(DoubleType())) \
                    .withColumn("Male_Population", df_spark["Male_Population"].cast(IntegerType())) \
                    .withColumn("Female_Population", df_spark["Female_Population"].cast(IntegerType())) \
                    .withColumn("Total_Population", df_spark["Total_Population"].cast(IntegerType())) \
                    .withColumn("Foreign_born", df_spark["Foreign_born"].cast(IntegerType())) \
                    .withColumn("Count", df_spark["Count"].cast(IntegerType()))

In [267]:
df_spark = df_spark.groupBy(['City', 'State', 'Median_Age', 'Male_Population',\
                             'Female_Population', 'Total_Population', 'Foreign_born', \
                                      'State_Code'])\
                            .pivot("Race")\
                            .sum("Count")

In [268]:
df_spark = df_spark.withColumnRenamed("American Indian and Alaska Native", "Native")\
                        .withColumnRenamed("Black or African-American", "Black")\
                        .withColumnRenamed("Hispanic or Latino", "Hispanic")

In [269]:
df_spark.createOrReplaceTempView("pivot_table")

In [270]:
dem_by_state = spark.sql("""
            SELECT State_Code, 
            cast ( avg(Median_Age) as DECIMAL(4,2) ) as age,
            sum(Male_Population) as Male_Population,
            sum(Female_Population) as Female_Population,
            sum(Total_Population) as Total_Population, 
            sum(Foreign_born) as Foreign_born,
            sum(Native) as Native,
            sum(Asian) as Asian,
            sum(Black) as Black,
            sum(White) as White
            FROM pivot_table
            GROUP BY State_Code
            """)

In [271]:
dem_by_state.show(5)

+----------+-----+---------------+-----------------+----------------+------------+------+------+------+-------+
|State_Code|  age|Male_Population|Female_Population|Total_Population|Foreign_born|Native| Asian| Black|  White|
+----------+-----+---------------+-----------------+----------------+------------+------+------+------+-------+
|        AZ|35.04|        2227455|          2272087|         4499542|      682313|129708|229183|296222|3591611|
|        SC|34.18|         260944|           272713|          533657|       27744|  3705| 13355|175064| 343764|
|        LA|34.63|         626998|           673597|         1300595|       83419|  8263| 38739|602377| 654578|
|        MN|35.62|         702157|           720246|         1422403|      215873| 25242|151544|216731|1050239|
|        NJ|35.12|         705736|           723172|         1428908|      477028| 11350|116844|452202| 615083|
+----------+-----+---------------+-----------------+----------------+------------+------+------+------+-

In [278]:
#dem_by_state.write.parquet(os.path.join("s3a://dend-capstone/","us_demographic"), mode="overwrite")

### Cleaning Steps
Document steps necessary to clean the data

#### I94 Immigration Data

1. I'll group data by countrt where imigrant comes from, and US state where immgirants enteres US
2. Remove all records where country (xxx) is Null
3. After understaning the dataset, I came up with list of columns that will server my project purpose as definded abov.
4. All undesired columns will be removed.
5. For the column (I94MODE), the codes will be replaced with the values.
6. For the column (I94VISA), the codes will be replaced with the values.

#### World Temperature Data

1. I'll filter dataset to include only US data
2. I'll filter dataset to include only > 2011 data or above.
3. I'll extract year and month from date
4. Convert to the appropriate data type
5. drop uneccly columns
6. repalce State names with abbrivation

#### U.S. City Demographic Data

#### Airport Code Table

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

In [1]:
spark.stop()

NameError: name 'spark' is not defined